In [38]:
import pandas as pd
from pandas.io.json import json_normalize 
import os
import plotly.express as px
import json 

dataset = "CIFAR100"
all_subdirs = [os.path.join(dataset,d) for d in os.listdir(dataset) if os.path.isdir(os.path.join(dataset, d)) and d != "cifar-100-python"]
print(all_subdirs)
#latest_folder = max(all_subdirs, key=os.path.getmtime)
latest_folder = os.path.join(dataset,"18-10-2020-17:53:30")

data = []
with open(os.path.join(latest_folder, "results.jsonl"), 'r', encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line))

#print(data)
df = json_normalize(data)
df = df.applymap(lambda x: x[0] if (isinstance(x, list) and len(x) == 1) else x)

def nice_name(row):
    if row["model"] == "GNCLClassifier":
        if row["l_reg"] == 0:
            return "Ind."
        else:
            return u"GNLC \u03BB = {}".format(row["l_reg"]) 
    elif row["model"] == "BaggingClassifier":
        return "Bagging"
    elif row["model"] == "E2EEnsembleClassifier":
        return "E2E"
    elif row["model"] == "SKLearnModel":
        return "Single"
    elif row["model"] == "SMCLClassifier":
        return "SMCL"
    elif row["model"] == "GradientBoostedNets":
        return "GB"
    elif row["model"] == "SnapshotEnsembleClassifier":
        return "SE"
    else:
        return row["model"]

def base_type(row):
    if "binary" in row["base_estimator"]:
        mtype = "binary"
    else:
        mtype = "float"

    if "small" in row["base_estimator"]:
        size = "small"
    else:
        size = "large"

    return row["base_estimator"].split("_")[0] + " " + size + " " + mtype

print(df.columns)     
df["nice_name"] = df.apply(nice_name, axis=1)
df["model_type"] = df.apply(base_type, axis=1)
df["complexity"] = df.apply(lambda x : x["scores.params_test"] if "float" in x["model_type"] else x["scores.params_test"] / 32.0, axis=1)
df["scores.avg_accurcay_test"] = df["scores.avg_accurcay_test"]
df["scores.accuracy_test"] = 100.0*df["scores.accuracy_test"]
df = df.round(decimals = 3)

print("{} experiments done".format(len(df)))
cols = df.columns
cols = ["nice_name", "model_type", "mode", "scores.accuracy_test", "scores.params_test", "scores.diversity_test"] + list(set(cols) - set(["nice_name", "model_type", "mode", "scores.accuracy_test", "scores.params_test", "scores.diversity_test"]))
df = df.reindex(cols, axis=1)
df
#dfbinary = df[df["model_type"] == "binary"]
#dffloat = df[df["model_type"] == "float"]  
#dffloat


['CIFAR100/13-10-2020-22:48:28', 'CIFAR100/04-11-2020-09:28:19', 'CIFAR100/13-10-2020-22:33:43', 'CIFAR100/13-10-2020-22:38:46', 'CIFAR100/17-10-2020-22:56:23', 'CIFAR100/17-10-2020-22:37:20', 'CIFAR100/15-10-2020-22:29:04', 'CIFAR100/13-10-2020-22:44:11', 'CIFAR100/04-11-2020-09:23:47', 'CIFAR100/31-10-2020-09:50:44', 'CIFAR100/13-10-2020-23:46:30', 'CIFAR100/13-10-2020-23:40:44', 'CIFAR100/13-10-2020-23:27:01', 'CIFAR100/13-10-2020-22:40:15', 'CIFAR100/13-10-2020-22:34:54', 'CIFAR100/13-10-2020-23:43:43', 'CIFAR100/15-10-2020-22:21:39', 'CIFAR100/13-10-2020-23:48:31', 'CIFAR100/17-10-2020-22:54:02', 'CIFAR100/13-10-2020-23:48:03', 'CIFAR100/13-10-2020-19:07:51', 'CIFAR100/13-10-2020-22:37:16', 'CIFAR100/13-10-2020-23:49:55', 'CIFAR100/13-10-2020-23:42:29', 'CIFAR100/13-10-2020-23:03:19', 'CIFAR100/15-10-2020-22:18:16', 'CIFAR100/13-10-2020-22:08:02', 'CIFAR100/04-11-2020-09:19:12', 'CIFAR100/04-11-2020-09:20:20', 'CIFAR100/15-10-2020-22:19:24', 'CIFAR100/04-11-2020-09:43:16', 'CIFAR1

,nice_name,model_type,mode,scores.accuracy_test,scores.params_test,scores.diversity_test,num_gpus,scores.diversity_train,complexity,train,...,transformer,scoring.avg_accurcay,out_path,verbose,combination_type,loss_function,data_loader,scoring.loss,scores.avg_accurcay_test,test
0,Single,simpleresnet small float,NaN,56.25,88100,0.000,1.0,0.000,88100.0,"[/data/s1/buschjae/CIFAR100/, False]",...,Compose(\n Normalize(mean=[-2.4290657439446...,avg_accurcay,/home/buschjae/projects/bnn/CIFAR100/18-10-202...,False,NaN,CrossEntropyLoss(),read_data,loss,0.000,"[/data/s1/buschjae/CIFAR100/, True]"
1,E2E,simpleresnet small float,NaN,67.26,1409600,107.196,1.0,20.758,1409600.0,"[/data/s1/buschjae/CIFAR100/, False]",...,Compose(\n Normalize(mean=[-2.4290657439446...,avg_accurcay,/home/buschjae/projects/bnn/CIFAR100/18-10-202...,False,NaN,CrossEntropyLoss(),read_data,loss,3.471,"[/data/s1/buschjae/CIFAR100/, True]"
2,SMCL,simpleresnet small float,NaN,30.31,1409600,5.637,1.0,5.875,1409600.0,"[/data/s1/buschjae/CIFAR100/, False]",...,Compose(\n Normalize(mean=[-2.4290657439446...,avg_accurcay,/home/buschjae/projects/bnn/CIFAR100/18-10-202...,False,best,CrossEntropyLoss(),read_data,loss,5.451,"[/data/s1/buschjae/CIFAR100/, True]"
3,Bagging,simpleresnet small float,NaN,61.62,1409600,0.522,1.0,0.478,1409600.0,"[/data/s1/buschjae/CIFAR100/, False]",...,Compose(\n Normalize(mean=[-2.4290657439446...,avg_accurcay,/home/buschjae/projects/bnn/CIFAR100/18-10-202...,False,NaN,CrossEntropyLoss(),read_data,loss,50.731,"[/data/s1/buschjae/CIFAR100/, True]"
4,Ind.,simpleresnet small float,upper,65.32,1409600,0.391,1.0,0.322,1409600.0,"[/data/s1/buschjae/CIFAR100/, False]",...,Compose(\n Normalize(mean=[-2.4290657439446...,avg_accurcay,/home/buschjae/projects/bnn/CIFAR100/18-10-202...,False,average,CrossEntropyLoss(),read_data,loss,56.137,"[/data/s1/buschjae/CIFAR100/, True]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,GB,simpleresnet large binary,NaN,59.47,11315792,0.780,NaN,0.795,353618.5,"[/data/s1/buschjae/CIFAR100/, False]",...,Compose(\n Normalize(mean=[-2.4290657439446...,avg_accurcay,/home/buschjae/projects/bnn/CIFAR100/31-10-202...,True,NaN,CrossEntropyLoss(),read_data,loss,4.849,"[/data/s1/buschjae/CIFAR100/, True]"
60,SE,simpleresnet small float,NaN,60.74,1409600,7.755,NaN,7.294,1409600.0,"[/data/s1/buschjae/CIFAR100/, False]",...,Compose(\n Normalize(mean=[-2.4290657439446...,avg_accurcay,/home/buschjae/projects/gncl/CIFAR100/04-11-20...,True,NaN,CrossEntropyLoss(),read_data,loss,16.334,"[/data/s1/buschjae/CIFAR100/, True]"
61,SE,simpleresnet small binary,NaN,54.44,1413712,12.433,NaN,12.220,44178.5,"[/data/s1/buschjae/CIFAR100/, False]",...,Compose(\n Normalize(mean=[-2.4290657439446...,avg_accurcay,/home/buschjae/projects/gncl/CIFAR100/04-11-20...,True,NaN,CrossEntropyLoss(),read_data,loss,9.203,"[/data/s1/buschjae/CIFAR100/, True]"
62,SE,simpleresnet large float,NaN,67.53,11303488,10.675,NaN,5.036,11303488.0,"[/data/s1/buschjae/CIFAR100/, False]",...,Compose(\n Normalize(mean=[-2.4290657439446...,avg_accurcay,/home/buschjae/projects/gncl/CIFAR100/04-11-20...,True,NaN,CrossEntropyLoss(),read_data,loss,20.246,"[/data/s1/buschjae/CIFAR100/, True]"


In [40]:
import numpy as np
import plotly.io as pio

if not os.path.exists("images"):
    os.mkdir("images") 

pio.orca.config.use_xvfb = True

n_estimators = [16]
for mtype in ["simpleresnet small float", "simpleresnet small binary", "simpleresnet large float", "simpleresnet large binary"]:
#mtype, n_estimators = "simpleresnet small float", [16]
#mtype, n_estimators = "simpleresnet small binary", [16]
#mtype, n_estimators = "simpleresnet large float", [16]
#mtype, n_estimators = "simpleresnet large binary", [16]

    methods_order = ["SMCL", "GB", "SE", "Bagging", "Ind.", "GNLC λ = 0.1", "GNLC λ = 0.2", "GNLC λ = 0.3", "GNLC λ = 0.4" ,"GNLC λ = 0.5", "GNLC λ = 0.6", "GNLC λ = 0.7", "GNLC λ = 0.8", "GNLC λ = 0.9", "E2E"]
    # mtype, n_estimators = "EfficientNetB0 float", [8, 16]
    #mtype, n_estimators = "EfficientNetB0 binary", [8, 16]

    filteredDF = df.loc[ df["model_type"] == mtype ]
    #n_estimators = sorted(set(filteredDF["n_estimators"].dropna()))

    for ne in n_estimators:
        dff = filteredDF.loc[ (filteredDF["n_estimators"] == ne)]
        accuracy_single = filteredDF.loc[filteredDF["n_estimators"].isnull()]["scores.accuracy_test"].to_numpy()[0]
        trainloss_single = filteredDF.loc[filteredDF["n_estimators"].isnull()]["scores.loss_train"].to_numpy()[0]
        #dff = dff.loc[ (dff["l_reg"] <= 0.5) | (dff["l_reg"].isnull())]

        #print(dff)
        #dff = dff.loc[ dff["l_Reg"] < 1000]
        #dff = df[ (df["model_type"] == mtype) and (df["n_estimators"] == ne or df["n_estimators"] == np.nan)]

        fig = px.bar(dff, x="nice_name",y="scores.accuracy_test",
            title="Test accuracy with T = {} and {} models".format(ne, mtype), 
            labels={
                "scores.accuracy_test": "Test accuracy",
                "nice_name": "Method"
            },
            text = "scores.accuracy_test"
        )
        fig.update_traces(textposition='outside')
        fig.update_layout(xaxis={'categoryorder':'array', "categoryarray": methods_order})
        fig.add_shape(type='line',x0=-0.5,y0=accuracy_single,x1=len(methods_order)-1+0.5,
            y1=accuracy_single,line=dict(color='Black',),xref='x',yref='y'
        )
        #fig.update_layout(xaxis={'categoryorder':'category ascending'})
        fig.show()
        fig.write_image("images/{}_{}_{}.pdf".format("Accuracy",mtype.replace(" ", "-"), ne))

        fig = px.bar(dff, x="nice_name",y="scores.diversity_test",
            title="Test diversity with T = {} and {} models".format(ne, mtype), 
            labels={
                "scores.diversity_test": "Diversity",
                "nice_name": "Method"
            },
            log_y = True,
            text = "scores.diversity_test"
        )
        fig.update_traces(textposition='outside')
        #fig.update_layout(xaxis={'categoryorder':'category ascending'})
        fig.update_layout(xaxis={'categoryorder':'array', "categoryarray": methods_order})
        fig.show()
        fig.write_image("images/{}_{}_{}.pdf".format("Diversity",mtype.replace(" ", "-"), ne))

        fig = px.bar(dff, x="nice_name",y="scores.avg_accurcay_test",
            title="Average test accuracy with T = {} and {} models".format(ne, mtype), 
            labels={
                "scores.avg_accurcay_test": "Average test accuracy of each ensemble member",
                "nice_name": "Method"
            },
            text = "scores.avg_accurcay_test"
        )
        fig.update_traces(textposition='outside')
        #fig.update_layout(xaxis={'categoryorder':'category ascending'})
        fig.update_layout(xaxis={'categoryorder':'array', "categoryarray": methods_order})
        fig.show()
        fig.write_image("images/{}_{}_{}.pdf".format("AvgAccuracy",mtype.replace(" ", "-"), ne))

        fig = px.bar(dff, x="nice_name",y="complexity",
            title="Model complexity with T = {} and {} models".format(ne, mtype), 
            labels={
                "complexity": "Model complexity (# of trainable parameters)",
                "nice_name": "Method"
            },
            text = "complexity"
        )
        fig.update_traces(textposition='outside')
        #fig.update_layout(xaxis={'categoryorder':'category ascending'})
        fig.update_layout(xaxis={'categoryorder':'array', "categoryarray": methods_order})
        fig.show()
        fig.write_image("images/{}_{}_{}.pdf".format("Complexity",mtype.replace(" ", "-"), ne))

        fig = px.bar(dff, x="nice_name",y="scores.loss_train",
            title="Training loss with T = {} and {} models".format(ne, mtype), 
            labels={
                "scores.loss_train": "Training loss",
                "nice_name": "Method"
            },
            log_y = True,
            text = "scores.loss_train"
        )
        fig.update_traces(textposition='outside')
        #fig.update_layout(xaxis={'categoryorder':'category ascending'})
        fig.update_layout(xaxis={'categoryorder':'array', "categoryarray": methods_order})
        fig.add_shape(type='line',x0=-0.5,y0=trainloss_single,x1=len(methods_order)-1+0.5,
            y1=trainloss_single,line=dict(color='Black',),xref='x',yref='y'
        )
        # TODO USE MAKE SUBPLOTS HERE
        fig.show()
        fig.write_image("images/{}_{}_{}.pdf".format("Loss",mtype.replace(" ", "-"), ne))
        



In [99]:
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio

fig = make_subplots(rows=4, cols=3, subplot_titles=["Small Binary", "Small Float", "Large Float"], horizontal_spacing = 0.03, vertical_spacing = 0.02)
    
methods_order = ["SMCL", "GB", "SE", "Bagging", "Ind.", "GNLC λ = 0.1", "GNLC λ = 0.2", "GNLC λ = 0.3", "GNLC λ = 0.4" ,"GNLC λ = 0.5", "GNLC λ = 0.6", "GNLC λ = 0.7", "GNLC λ = 0.8", "GNLC λ = 0.9", "E2E"]

shapes = []
for i, mtype in enumerate(["simpleresnet small binary", "simpleresnet small float", "simpleresnet large float"]):
    filteredDF = df.loc[ df["model_type"] == mtype ]
    dff = filteredDF.loc[ (filteredDF["n_estimators"] == 16)]
    accuracy_single = filteredDF.loc[filteredDF["n_estimators"].isnull()]["scores.accuracy_test"].to_numpy()[0]
    trainloss_single = filteredDF.loc[filteredDF["n_estimators"].isnull()]["scores.loss_train"].to_numpy()[0]

    fig.add_trace(go.Bar(x = dff["nice_name"], y = dff["scores.accuracy_test"], texttemplate='%{text:.2f}', textposition='inside', text=dff["scores.accuracy_test"], showlegend=False, marker_color="#636EFA", textfont_size=14, textangle=0),row=1,col=i+1)
    fig.add_trace(go.Bar(x = dff["nice_name"], y = dff["scores.diversity_test"], texttemplate='%{text:.2f}', textposition='inside', text=dff["scores.diversity_test"], showlegend=False, marker_color="#636EFA", textfont_size=14, textangle=0),row=2,col=i+1)
    fig.add_trace(go.Bar(x = dff["nice_name"], y = dff["scores.avg_accurcay_test"], texttemplate='%{text:.2f}', textposition='inside', text=dff["scores.avg_accurcay_test"], showlegend=False, marker_color="#636EFA", textfont_size=14, textangle=0),row=3,col=i+1)
    fig.add_trace(go.Bar(x = dff["nice_name"], y = dff["scores.loss_train"], texttemplate='%{text:.2f}', textposition='inside', text=dff["scores.loss_train"], showlegend=False, marker_color="#636EFA", textfont_size=14, textangle=0),row=4,col=i+1)
    
    fig.update_yaxes(row=1, col=i+1, dtick=10, tickmode = 'linear',tick0 = 0)
    fig.update_yaxes(row=2, col=i+1, type="log")
    fig.update_yaxes(row=3, col=i+1, dtick=10, tickmode = 'linear',tick0 = 0)
    fig.update_yaxes(row=4, col=i+1, type="log")
        
    fig.update_xaxes(row=1, col=i+1, categoryarray=methods_order, showticklabels=False)
    fig.update_xaxes(row=2, col=i+1, categoryarray=methods_order, showticklabels=False)
    fig.update_xaxes(row=3, col=i+1, categoryarray=methods_order, showticklabels=False)
    fig.update_xaxes(row=4, col=i+1, categoryarray=methods_order, tickangle=45)
    
    shapes.extend([
        dict(
            type='line',xref='x'+str(i+1),yref='y'+str(i+1),x0=-0.5,y0=accuracy_single,x1=len(methods_order)-1+0.5,y1=accuracy_single,
            line=dict(color='Dark Grey',width=1,dash="dashdot")
        ),
        dict(
            type='line',xref='x'+str(i+10),yref='y'+str(i+10),x0=-0.5,y0=trainloss_single,x1=len(methods_order)-1+0.5,y1=trainloss_single,
            line=dict(color='Dark Grey',width=1,dash="dashdot")
        )
    ])
    

fig.update_yaxes(title_text="Accuracy", row=1, col=1, title_standoff = 25, title_font = {"size": 14})
fig.update_yaxes(title_text="Diversity", row=2, col=1, title_font = {"size": 14})
fig.update_yaxes(title_text="Average accuracy", row=3, col=1, title_font = {"size": 14})
fig.update_yaxes(title_text="Train loss", row=4, col=1, title_font = {"size": 14})


fig.update_layout(
    shapes=shapes,
    legend=dict(orientation="h",yanchor="bottom",y=-0.1,xanchor="left",x=0.1),
    margin={'l': 5, 'r': 20, 't': 20, 'b': 5},
    height=700, width=1200
)

fig.write_image("images/cifar100.pdf")
fig.show()
